In [761]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

In [762]:
# import libraries
from matplotlib import pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch as t
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import pandas as pd


#Seed 1
#torch.manual_seed(299)
#random.seed(53)
#np.random.seed(100)

#Seed 2
#torch.manual_seed(798)
#random.seed(50)
#np.random.seed(20)

#Seed 3
torch.manual_seed(500)
random.seed(55)
np.random.seed(9)



torch.use_deterministic_algorithms(True)

In [763]:
from torch.utils.data.dataset import TensorDataset
# choose the training and test datasets
iris_train_load=pd.read_csv('/content/iris_training.csv',header=None)
iris_train_load


input=iris_train_load.iloc[:,:4]
print('\nInput values are:') 
print(input.head())

output=iris_train_load.iloc[:,4:5]
print('\nThe output value is:') 
print(output.head()) 


# Convert Input and Output data to Tensors and create a TensorDataset 
input = torch.Tensor(input.to_numpy())      # Create tensor of type torch.float32 
print('\nInput format: ', input.shape, input.dtype)     # Input format: torch.Size([120, 4]) torch.float32 
output = torch.tensor(output.to_numpy())        # Create tensor type torch.int64  
print('Output format: ', output.shape, output.dtype)  # Output format: torch.Size([120, 1]) torch.int64 
data_train = TensorDataset(input, output)  





iris_test_load=pd.read_csv('/content/iris_test.csv',header=None)
input_test=iris_test_load.iloc[:,:4]
print('\nInput values are:') 
print(input_test.head())

output_test=iris_test_load.iloc[:,4:5]
print('\nThe output value is:') 
print(output_test.head()) 


# Convert Input and Output data to Tensors and create a TensorDataset 
input_test = torch.Tensor(input_test.to_numpy())      # Create tensor of type torch.float32 
print('\nInput format: ', input_test.shape, input_test.dtype)     # Input format: torch.Size([30, 4]) torch.float32 
output_test = torch.tensor(output_test.to_numpy())        # Create tensor type torch.int64  
print('Output format: ', output_test.shape, output_test.dtype)  # Output format: torch.Size([30, 1]) torch.int64 
data_test = TensorDataset(input_test, output_test)  




Input values are:
     0    1    2    3
0  6.4  2.8  5.6  2.2
1  5.0  2.3  3.3  1.0
2  4.9  2.5  4.5  1.7
3  4.9  3.1  1.5  0.1
4  5.7  3.8  1.7  0.3

The output value is:
   4
0  2
1  1
2  2
3  0
4  0

Input format:  torch.Size([120, 4]) torch.float32
Output format:  torch.Size([120, 1]) torch.int64

Input values are:
     0    1    2    3
0  5.9  3.0  4.2  1.5
1  6.9  3.1  5.4  2.1
2  5.1  3.3  1.7  0.5
3  6.0  3.4  4.5  1.6
4  5.5  2.5  4.0  1.3

The output value is:
   4
0  1
1  2
2  0
3  1
4  1

Input format:  torch.Size([30, 4]) torch.float32
Output format:  torch.Size([30, 1]) torch.int64


In [764]:
train_batch_size = 20
number_rows = len(input)    # The size of our dataset or the number of rows in excel table.  
val_split = int(number_rows*0.3)  
train_split = number_rows - val_split   
train_set, validate_set= torch.utils.data.random_split( 
    data_train, [train_split, val_split])    
 
# Create Dataloader to read the data within batch sizes and put into memory. 
train_loader = torch.utils.data.DataLoader(train_set,batch_size=train_batch_size ) 
validate_loader = torch.utils.data.DataLoader(validate_set,batch_size=1) 
test_loader = torch.utils.data.DataLoader(data_test,batch_size=1)

In [765]:
## Define the NN architecture

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 3) 
       

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #output = F.softmax(x,dim=1)
        return x


# initialize the NN
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')
model = Net().to(device)
print(model)

Net(
  (fc1): Linear(in_features=4, out_features=3, bias=True)
)


In [766]:
## Specify loss and optimization functions

# specify loss function
criterion = nn.CrossEntropyLoss().to(device)

# specify optimizer
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.09)

print(criterion, optimizer)

CrossEntropyLoss() Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.09
    lr_decay: 0
    weight_decay: 0
)


In [767]:
# Training Function 
def train(num_epochs): 
    best_accuracy = 0.0 
     
    print("Begin training...") 
    for epoch in range(1, num_epochs+1): 
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_vall_loss = 0.0 
        total = 0 
 
        # Training Loop 
        for data in train_loader: 
        #for data in enumerate(train_loader, 0): 
            inputs, outputs = data  # get the input and real species as outputs; data is a list of [inputs, outputs] 
            optimizer.zero_grad()   # zero the parameter gradients          
            predicted_outputs = model(inputs)   # predict output from the model 
            train_loss = criterion(predicted_outputs, torch.flatten(outputs))   # calculate loss for the predicted output  
            train_loss.backward()   # backpropagate the loss 
            optimizer.step()        # adjust parameters based on the calculated gradients 
            running_train_loss +=train_loss.item()  # track the loss value 
 
        # Calculate training loss value 
        train_loss_value = running_train_loss/len(train_loader) 
 
        # Validation Loop 
        with torch.no_grad(): 
          model.eval() 
          for data in validate_loader: 
            inputs, outputs = data 
            predicted_outputs = model(inputs) 
            val_loss = criterion(predicted_outputs, torch.flatten(outputs)) 
             
            # The label with the highest value will be our prediction 
            _, predicted = torch.max(predicted_outputs, 1)
            running_vall_loss += val_loss.item()  
            total += outputs.size(0) 
            running_accuracy += (predicted.eq(torch.flatten(outputs))).sum().item() 

 
        # Calculate validation loss value 
        val_loss_value = running_vall_loss/len(validate_loader) 
                
        # Calculate accuracy as the number of correct predictions in the validation batch divided by the total number of predictions done.  
        accuracy = (100 * running_accuracy / total)     

         
        # Print the statistics of the epoch 
        print('Completed training batch', epoch, 'Training Loss is: %.4f' %train_loss_value, 'Validation Loss is: %.4f' %val_loss_value, 'Accuracy is %d %%' % (accuracy))

In [768]:
# Function to test the model 
def test(): 
    total=0.0
    running_accuracy=0.0
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            outputs = outputs.to(torch.float32) 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
            total += outputs.size(0) 
            running_accuracy += (predicted == torch.flatten(outputs)).sum().item() 
 
        print('Test Accuracy is: %d %%' % (100 * running_accuracy / total))   

In [769]:
if __name__ == "__main__": 
    num_epochs = 1000
    train(num_epochs) 
    print('Finished Training\n') 
    test() 


Begin training...
Completed training batch 1 Training Loss is: 1.2560 Validation Loss is: 0.8376 Accuracy is 69 %
Completed training batch 2 Training Loss is: 0.7869 Validation Loss is: 0.7429 Accuracy is 69 %
Completed training batch 3 Training Loss is: 0.7045 Validation Loss is: 0.6912 Accuracy is 69 %
Completed training batch 4 Training Loss is: 0.6554 Validation Loss is: 0.6555 Accuracy is 69 %
Completed training batch 5 Training Loss is: 0.6209 Validation Loss is: 0.6286 Accuracy is 69 %
Completed training batch 6 Training Loss is: 0.5947 Validation Loss is: 0.6072 Accuracy is 69 %
Completed training batch 7 Training Loss is: 0.5738 Validation Loss is: 0.5896 Accuracy is 69 %
Completed training batch 8 Training Loss is: 0.5566 Validation Loss is: 0.5754 Accuracy is 69 %
Completed training batch 9 Training Loss is: 0.5422 Validation Loss is: 0.5629 Accuracy is 69 %
Completed training batch 10 Training Loss is: 0.5297 Validation Loss is: 0.5516 Accuracy is 69 %
Completed training ba